In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [4]:
data = data.loc[data['Year']== '2017']
data

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
16050,18285,Don't Smoke Rock,"Smoke DZA,Pete Rock",don’t take your eyes off pete rock. the early-...,Babygrande,7.4,January,2,2017,Rap,NaN,NaN,NaN
16053,18288,December 99th,Yasiin Bey,"in january 2016, rapper/actor yasiin bey annou...","A Country Called Earth,AWGE",3.5,January,2,2017,Rap,NaN,NaN,NaN
16054,18289,Nadir,Steven Warwick,"“why so sad?/don’t feel so bad/get out of bed,...",Pan,7.6,January,3,2017,Electronic,NaN,NaN,NaN
16055,18290,"The Fragile (2017 Definitive Edition),The Frag...","Nine Inch Nails,Nine Inch Nails","when , debuted ,, the followup to his star-mak...","The Null Corporation,The Null Corporation,The ...","8.7,6.6",January,11,2017,Rock,NaN,NaN,NaN
16057,18292,White Friday (CM9),Yo Gotti,"has long been memphis’ biggest rap star, but ...",Epic,6.8,January,11,2017,Rap,NaN,NaN,NaN
16058,18293,50,Michael Chapman,"serves a number of roles in ,’s gargantuan ca...",Paradise of Bachelors,7.7,January,14,2017,Rock,NaN,NaN,NaN
16061,18296,Of Murk and Shallow Water EP,9T Antiope,", the duo of nima aghiani and sara bigdeli sha...",Self-released,7.8,January,11,2017,Electronic,NaN,NaN,NaN
16062,18297,Sferic Ghost Transmits,"Shackleton,Vengeance Tenfold",among club artists as well as experimental com...,Honest Jon's,8.0,January,21,2017,Electronic,NaN,NaN,NaN
16063,18298,Migration,Bonobo,"it’s been a long road for simon green, aka ,, ...",Ninja Tune,7.3,January,14,2017,Electronic,Jazz,NaN,NaN
16064,18299,Twin Peaks: Fire Walk With Me OST,Angelo Badalamenti,despite a career that stretches back a half-ce...,Death Waltz,8.4,January,20,2017,Experimental,NaN,NaN,NaN


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data['content']))

print(data_words[:1])

[['don', 'take', 'your', 'eyes', 'off', 'pete', 'rock', 'the', 'early', 'albums', 'he', 'produced', 'with', 'rapper', 'cl', 'smooth', 'still', 'live', 'in', 'legend', 'the', 'new', 'york', 'duo', 'are', 'reuniting', 'for', 'their', 'first', 'new', 'music', 'since', 'nas', 'dropped', 'this', 'pair', 'like', 'their', 'contemporaries', 'tribe', 'called', 'quest', 'weren', 'about', 'to', 'go', 'back', 'into', 'the', 'studio', 'just', 'because', 'fans', 'wanted', 'them', 'to', 'rock', 'may', 'be', 'the', 'golden', 'age', 'god', 'whose', 'snap', 'crack', 'boom', 'bap', 'helped', 'spawn', 'dilla', 'mark', 'ronson', 'and', 'kanye', 'west', 'but', 'take', 'look', 'at', 'the', 'names', 'that', 'make', 'up', 'rock', 'recent', 'client', 'list', 'torae', 'grafh', 'snyp', 'life', 'and', 'it', 'clear', 'that', 'he', 'less', 'interested', 'in', 'collecting', 'checks', 'than', 'he', 'is', 'crowding', 'his', 'orbit', 'with', 'hard', 'nosed', 'hip', 'hop', 'on', 'rock', 'first', 'full', 'length', 'collab

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['don', 'take', 'your', 'eyes', 'off', 'pete', 'rock', 'the', 'early', 'albums', 'he', 'produced', 'with', 'rapper', 'cl', 'smooth', 'still', 'live', 'in', 'legend', 'the', 'new_york', 'duo', 'are', 'reuniting', 'for', 'their', 'first', 'new', 'music', 'since', 'nas', 'dropped', 'this', 'pair', 'like', 'their', 'contemporaries', 'tribe', 'called', 'quest', 'weren', 'about', 'to', 'go', 'back', 'into', 'the', 'studio', 'just', 'because', 'fans', 'wanted', 'them', 'to', 'rock', 'may', 'be', 'the', 'golden_age', 'god', 'whose', 'snap', 'crack', 'boom_bap', 'helped', 'spawn', 'dilla', 'mark', 'ronson', 'and', 'kanye_west', 'but', 'take', 'look', 'at', 'the', 'names', 'that', 'make', 'up', 'rock', 'recent', 'client', 'list', 'torae', 'grafh', 'snyp', 'life', 'and', 'it', 'clear', 'that', 'he', 'less', 'interested', 'in', 'collecting', 'checks', 'than', 'he', 'is', 'crowding', 'his', 'orbit', 'with', 'hard_nosed', 'hip_hop', 'on', 'rock', 'first', 'full_length', 'collaboration', 'since', 'wi

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['take', 'eye', 'pete', 'rock', 'early', 'album', 'produce', 'rapper', 'cl', 'smooth', 'still', 'live', 'legend', 'new_york', 'duo', 'reunite', 'first', 'new', 'music', 'drop', 'pair', 'contemporary', 'tribe', 'call', 'quest', 'go', 'back', 'studio', 'fan', 'want', 'rock', 'may', 'golden_age', 'god', 'whose', 'snap', 'crack', 'boom_bap', 'help', 'spawn', 'mark', 'ronson', 'kanye_w', 'take', 'look', 'name', 'make', 'rock', 'recent', 'client', 'list', 'torae', 'grafh', 'snyp', 'life', 'clear', 'less', 'interested', 'collecting', 'check', 'crowd', 'orbit', 'hard_nos', 'rock', 'first', 'full_length', 'collaboration', 'smif', 'anoint', 'partner', 'smoke', 'dza', 'album', 'play', 'wintertime', 'drive', 'harlem', 'bentley', 'black', 'tint', 'window', 'swerve', 'together', 'pure', 'confidence', 'especially', 'suit', 'dza', 'prolific', 'working', 'blunt', 'flow', 'point', 'storytelling', 'skill', 'struggle', 'distinguish', 'crowded', 'world', 'nyc', 'street', 'rap', 'revivalist', 'though', 'ma

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 3), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 2), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 2), (108, 2), (109, 1), (110, 1)

In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.079*"metal" + 0.068*"smith" + 0.039*"guitarist" + 0.031*"drummer" + '
  '0.026*"bassist" + 0.021*"noise" + 0.021*"hardcore" + 0.021*"thrash" + '
  '0.018*"doom" + 0.015*"goth"'),
 (1,
  '0.013*"woman" + 0.008*"romance" + 0.008*"gay" + 0.008*"bad" + '
  '0.008*"someone" + 0.007*"british" + 0.007*"songwriter" + 0.007*"beam" + '
  '0.007*"true" + 0.007*"largely"'),
 (2,
  '0.154*"punk" + 0.042*"hardcore" + 0.021*"reject" + 0.018*"judge" + '
  '0.017*"rev" + 0.017*"ridiculous" + 0.017*"portray" + 0.016*"absurdity" + '
  '0.016*"bus" + 0.016*"laugh"'),
 (3,
  '0.079*"folk" + 0.048*"topic" + 0.037*"implication" + 0.035*"lucky" + '
  '0.035*"pedal_steel" + 0.034*"defy" + 0.033*"sixth" + 0.032*"americana" + '
  '0.026*"agent" + 0.024*"banjo"'),
 (4,
  '0.004*"american" + 0.004*"future" + 0.004*"radio" + 0.003*"member" + '
  '0.003*"actually" + 0.003*"especially" + 0.003*"past" + 0.003*"white" + '
  '0.003*"country" + 0.003*"hope"'),
 (5,
  '0.004*"soul" + 0.004*"death" + 0.004*"story

In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.10183002046432

Coherence Score:  0.41293657313952775


In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.075202  0.008125       1        1  27.155172
6      0.010651  0.186624       2        1  25.213068
4      0.114575  0.029734       3        1  21.891268
8      0.036710  0.212087       4        1  16.111063
1      0.134508 -0.183624       5        1   4.230605
7      0.184148 -0.187110       6        1   3.453146
0     -0.296088 -0.101373       7        1   1.110976
2     -0.201224 -0.095109       8        1   0.590152
3     -0.036401  0.063204       9        1   0.171185
9     -0.022084  0.067441      10        1   0.073364, topic_info=     Category        Freq           Term       Total  loglift  logprob
1054  Default  357.000000           punk  357.000000  30.0000  30.0000
94    Default  368.000000         rapper  368.000000  29.0000  29.0000
876   Default  184.000000           folk  184.000000  28.0000  28.0000
555   Default  229.000000      guitarist  229.000000  27.0000  27.0000
1155  Default  297.000000          death  297.000000  26.0000  26.0000
959   Default  202.000000          metal  202.000000  25.0000  25.0000
1158  Default  220.000000        drummer  220.000000  24.0000  24.0000
1632  Default  245.000000          noise  245.000000  23.0000  23.0000
4135  Default  175.000000          smith  175.000000  22.0000  22.0000
138   Default  311.000000          verse  311.000000  21.0000  21.0000
1318  Default  314.000000           city  314.000000  20.0000  20.0000
1401  Default  258.000000          heavy  258.000000  19.0000  19.0000
1928  Default  297.000000          woman  297.000000  18.0000  18.0000
1138  Default  158.000000        bassist  158.000000  17.0000  17.0000
943   Default  357.000000     electronic  357.000000  16.0000  16.0000
53    Default  236.000000           flow  236.000000  15.0000  15.0000
100   Default  280.000000         sample  280.000000  14.0000  14.0000
329   Default  159.000000        mixtape  159.000000  13.0000  13.0000
691   Default  140.000000         scream  140.000000  12.0000  12.0000
950   Default  286.000000         future  286.000000  11.0000  11.0000
2857  Default  200.000000           fuck  200.000000  10.0000  10.0000
1131  Default  156.000000       vocalist  156.000000   9.0000   9.0000
2140  Default  134.000000           fire  134.000000   8.0000   8.0000
1641  Default  242.000000          prove  242.000000   7.0000   7.0000
845   Default  176.000000           trap  176.000000   6.0000   6.0000
1690  Default  111.000000       hardcore  111.000000   5.0000   5.0000
553   Default  312.000000         groove  312.000000   4.0000   4.0000
231   Default  247.000000        someone  247.000000   3.0000   3.0000
4     Default  248.000000            bad  248.000000   2.0000   2.0000
118   Default  175.000000         street  175.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
2649  Topic10    0.028606      cacophony   10.355432   1.3258  -8.6858
3748  Topic10    0.028605          titan   10.435191   1.3181  -8.6858
3041  Topic10    0.028605    accommodate   10.498251   1.3121  -8.6858
3647  Topic10    0.028605       bullshit   10.603903   1.3021  -8.6858
3287  Topic10    0.028606   aggressively   10.672689   1.2957  -8.6857
2070  Topic10    0.028605         purity   10.765267   1.2870  -8.6858
2439  Topic10    0.028606         quirky   10.886539   1.2758  -8.6858
3824  Topic10    0.028623        compete   26.763222   0.3769  -8.6852
3650  Topic10    0.028617            gay   78.308662  -0.6969  -8.6854
4162  Topic10    0.028616     undeniably   16.338127   0.8702  -8.6854
2038  Topic10    0.028616         liquid   29.593393   0.2762  -8.6854
4060  Topic10    0.028616         offset   44.660065  -0.1354  -8.6854
3057  Topic10    0.028616        implore   20.230362   0.6565  -8.6854
791   Topic10    0.028616          emcee   26.978161   0.3687  -8.6854
3350  Topic10    0.028615        meeting   